In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         247 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  python_abi         conda-forge/linux-64::python_abi-3.7-2_cp37m

The following p

## Data sets
The data that will be used in this solution is from a json file available at https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json. The json file has the data related to all neighborhoods in the city of NewYork.

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In this solution we will focus on the Neighborhoods in Manhattan and Brooklyn. So, we will create separate dataframes containing the neighborhoods of Manhattan and Brooklyn

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

## Visualizing the neighborhoods of Manhattan on a map

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [13]:
brooklyn_address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(brooklyn_address)
brooklyn_latitude = location.latitude
brooklyn_longitude = location.longitude
print('The geograpical coordinate of Brroklyn are {}, {}.'.format(brooklyn_latitude, brooklyn_longitude))

The geograpical coordinate of Brroklyn are 40.6501038, -73.9495823.


In [14]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[brooklyn_latitude, brooklyn_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

## Explore the neighborhoods of Manhattan using Foursquare API

In [15]:
CLIENT_ID = 'CI0BLQRJ02X1AE5DW4PRV33FHB5MZAHUA5FI0VF4AQTRA4QJ' # your Foursquare ID
CLIENT_SECRET = 'ZM5SDB5EQRRHWEMROYOR3VHUW4FOIFHM00NU3Z5R3V5DQ2EC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)   
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [18]:
print('There are {} venues in Manhattan.'.format(manhattan_venues.shape[0]))

There are 3252 venues in Manhattan.


## Explore the neighborhoods of Brooklyn using Foursquare API

In [19]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [20]:
print('There are {} venues in Brooklyn.'.format(brooklyn_venues.shape[0]))

There are 2732 venues in Brooklyn.


## Filter the avenues in both Manhattan and Brooklyn to get the restaurants data

In [21]:
def asianrestaurantcount(row):
    neighborhood=row['Neighborhood']
    restaurants_neighborhood=restaurants_df[restaurants_df['Neighborhood']==neighborhood]
    asianrestaurants_neighborhood=restaurants_neighborhood[restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    return asianrestaurants_neighborhood.shape[0]

In [22]:
def brooklyn_asianrestaurantcount(row):
    neighborhood=row['Neighborhood']
    brooklyn_restaurants_neighborhood=brooklyn_restaurants_df[brooklyn_restaurants_df['Neighborhood']==neighborhood]
    brooklyn_asianrestaurants_neighborhood=brooklyn_restaurants_neighborhood[brooklyn_restaurants_neighborhood['Venue Category']=='Asian Restaurant']
    return brooklyn_asianrestaurants_neighborhood.shape[0]

In [23]:
restaurants_df=manhattan_venues[manhattan_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
restaurants=restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Manhattan is {}'.format(restaurants['Restaurant count'].sum()))

Total number of restarunats in Manhattan is 925


In [24]:
brooklyn_restaurants_df=brooklyn_venues[brooklyn_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
brooklyn_restaurants=brooklyn_restaurants_df.groupby(['Neighborhood']).size().reset_index(name='Restaurant count')
print('Total number of restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Restaurant count'].sum()))

Total number of restarunats in Brooklyn is 622


Manhattan has more number of restaurants than Brooklyn. Now, lets further filter the restaurants to find the number of Asian Restaurants.

In [25]:
restaurants['Asian Restaurant count']=restaurants.apply(asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Manhattan is {}'.format(restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Manhattan is 16


In [26]:
brooklyn_restaurants['Asian Restaurant count']=brooklyn_restaurants.apply(brooklyn_asianrestaurantcount,axis=1)
print('Total number of Asian restarunats in Brooklyn is {}'.format(brooklyn_restaurants['Asian Restaurant count'].sum()))

Total number of Asian restarunats in Brooklyn is 12


Though Manhattan has more restaurants than Brooklyn, it has fewer Asian restaurants than Brooklyn. So, Manhattan appears to be better option to open a Asian restaurant
## Lets dig in a little deeper to see the count of restaurants and Asian restaurants in each of the neighborhoods of Manhattan.
Count of restaurants and Asian restaurants in Manhattan grouped by Neighborhood

In [27]:
restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
12,Greenwich Village,44,0
36,Upper West Side,38,2
8,East Village,38,0
4,Chinatown,38,2
34,Turtle Bay,37,1
38,West Village,35,0
10,Flatiron,35,0
27,Noho,34,2
26,Murray Hill,33,1
31,Sutton Place,31,1


From the above summary, Greenwich Village and Chinatown have the most number of restaurant among the neighborhoods. In general, more number of restaurants implies that there is good demand for restaurants in that locality. Lets further study the neighborhoods of Greenwich Village and Chinatown to zero down on the best location for an Asian Restaurant

In [28]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
print(manhattan_onehot.shape)
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
print(manhattan_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

(3252, 334)
(3252, 335)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Ship

In [29]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

### Let's study each neighborhood in Manhattan to know the top 5 most common venues in that neighborhood.

In [31]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.10
1     Coffee Shop  0.07
2  Clothing Store  0.06
3           Hotel  0.04
4             Gym  0.03


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2    Wine Shop  0.04
3  Yoga Studio  0.03
4  Pizza Place  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1    French Restaurant  0.04
2  American Restaurant  0.04
3           Public Art  0.04
4   Chinese Restaurant  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1         Art Gallery  0.05
2              Bakery  0.05
3      Ice Cream Shop  0.03
4  Seafood Restaurant  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1               Bakery  0.08
2         Cocktail Bar  0.05
3  American Restaurant  0.04
4         Dessert Shop  0.04


----Civic Center----
                  venue  freq
0           Coffee Shop  0.07
1  Gym / Fitness C

We can see that both Greenwich Village and Chinatown have restaurants as the 4 out of 5 most common venues in the neighborhood. But, the restaurants in Chinatown appear to be those serving cuisines which overlap with Asian cuisine. Whereas, the restaurants in Greenwich Village as more spread out in terms of the cuisine they offer.

Further, Chitatown already has 2 Asian restaurants whereas Greenwich Village doesn't have any. So, an Asian restaurant would be a good addition to the restaurants available in it.

Let's study further to see if we can find any reason to support our inference that Greenwich Village is a good location to open an Asian restaurant.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Clothing Store,Hotel,Women's Store,Gym,Boat or Ferry,Memorial Site,Playground,Plaza
1,Carnegie Hill,Coffee Shop,Café,Wine Shop,Yoga Studio,Gym / Fitness Center,Pizza Place,Bookstore,French Restaurant,Cosmetics Shop,Bar
2,Central Harlem,African Restaurant,Public Art,Gym / Fitness Center,Chinese Restaurant,French Restaurant,American Restaurant,Seafood Restaurant,Bar,Beer Bar,Event Space
3,Chelsea,Coffee Shop,Bakery,Art Gallery,Wine Shop,Hotel,American Restaurant,Ice Cream Shop,French Restaurant,Seafood Restaurant,Market
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Salon / Barbershop,Optical Shop,Supermarket,Bar,Dumpling Restaurant
5,Civic Center,Coffee Shop,Spa,Gym / Fitness Center,American Restaurant,French Restaurant,Yoga Studio,Cocktail Bar,Italian Restaurant,Park,Hotel
6,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,American Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Cocktail Bar,Wine Shop,Spa
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Sandwich Place,Latin American Restaurant,Deli / Bodega,Cocktail Bar,Grocery Store,Gym,Seafood Restaurant
8,East Village,Bar,Speakeasy,Mexican Restaurant,Pizza Place,Vietnamese Restaurant,Cocktail Bar,Korean Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Italian Restaurant
9,Financial District,Coffee Shop,Pizza Place,Park,Cocktail Bar,Bar,Event Space,American Restaurant,Steakhouse,Italian Restaurant,Jewelry Store


In [34]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 1, 1, 3, 0, 1], dtype=int32)

In [35]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Coffee Shop,Sandwich Place,Gym,Yoga Studio,Bank,Department Store,Diner,Discount Store,Pharmacy,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Salon / Barbershop,Optical Shop,Supermarket,Bar,Dumpling Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Mobile Phone Shop,Bank,Gym,Latin American Restaurant,Supplement Shop,Spanish Restaurant,Tapas Restaurant,Sandwich Place
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Restaurant,Bakery,Café,Park,Chinese Restaurant,Lounge,Caribbean Restaurant,Pizza Place,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Deli / Bodega,Café,Mexican Restaurant,Coffee Shop,Yoga Studio,Bakery,Latin American Restaurant,Liquor Store,Park


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Salon / Barbershop,Optical Shop,Supermarket,Bar,Dumpling Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Bank,Gym,Latin American Restaurant,Supplement Shop,Spanish Restaurant,Tapas Restaurant,Sandwich Place
3,Inwood,Mexican Restaurant,Restaurant,Bakery,Café,Park,Chinese Restaurant,Lounge,Caribbean Restaurant,Pizza Place,Deli / Bodega
4,Hamilton Heights,Pizza Place,Deli / Bodega,Café,Mexican Restaurant,Coffee Shop,Yoga Studio,Bakery,Latin American Restaurant,Liquor Store,Park
8,Upper East Side,Exhibit,Italian Restaurant,American Restaurant,Juice Bar,Bakery,Gym / Fitness Center,Coffee Shop,Yoga Studio,Wine Shop,Hotel
11,Roosevelt Island,Park,Coffee Shop,Residential Building (Apartment / Condo),Liquor Store,Baseball Field,Outdoors & Recreation,Gym / Fitness Center,Gym,Greek Restaurant,Deli / Bodega
12,Upper West Side,Italian Restaurant,Bakery,Wine Bar,Bar,Café,Coffee Shop,Mediterranean Restaurant,Dessert Shop,Ice Cream Shop,French Restaurant
16,Murray Hill,Coffee Shop,Hotel,Japanese Restaurant,Sandwich Place,American Restaurant,Sushi Restaurant,Taco Place,Bar,Gym / Fitness Center,Burger Joint
18,Greenwich Village,Italian Restaurant,Clothing Store,Boutique,Sushi Restaurant,Coffee Shop,Dessert Shop,Cosmetics Shop,Indian Restaurant,Vietnamese Restaurant,Sandwich Place
19,East Village,Bar,Speakeasy,Mexican Restaurant,Pizza Place,Vietnamese Restaurant,Cocktail Bar,Korean Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Italian Restaurant


### The neighborhoods in cluster 0 predominantly have eatery joints as the top 10 common venues. Greenwich Village is also in this cluster, this further strengths our inference that it is a good option to open an Asian restaurant.
### The added advantage for Greenwich village is it is in close proximity of affluent neighborhoods such as Soho and Tribeca. So, it is quite possible that people of these affluent neighborhoods would visit a restaurant that serves a new cuisine in a nearby locality.
### Based on the above findings, we can conclude that Greenwich village is the best neighborhood among the neighborhoods of Manhattan and Brooklyn to open an Asian Restaurant.

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Deli / Bodega,Chinese Restaurant,Seafood Restaurant,Bar,Mexican Restaurant,Italian Restaurant,Cuban Restaurant,Supermarket,Sushi Restaurant
6,Central Harlem,African Restaurant,Public Art,Gym / Fitness Center,Chinese Restaurant,French Restaurant,American Restaurant,Seafood Restaurant,Bar,Beer Bar,Event Space
9,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Deli / Bodega,Sushi Restaurant,Ice Cream Shop,Wine Shop,Japanese Restaurant,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Cocktail Bar,Pizza Place,Gym,Burger Joint,Gym / Fitness Center,Cycle Studio
14,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,American Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Cocktail Bar,Wine Shop,Spa
23,Soho,Clothing Store,Boutique,Italian Restaurant,Shoe Store,Mediterranean Restaurant,Sporting Goods Shop,Bakery,Hotel,Coffee Shop,Women's Store
24,West Village,Italian Restaurant,New American Restaurant,American Restaurant,Park,Cocktail Bar,Wine Bar,Ice Cream Shop,Coffee Shop,Jazz Club,Chinese Restaurant
26,Morningside Heights,Coffee Shop,American Restaurant,Park,Bookstore,Café,Deli / Bodega,Sandwich Place,Burger Joint,Ethiopian Restaurant,Mediterranean Restaurant
35,Turtle Bay,Italian Restaurant,Coffee Shop,Sushi Restaurant,Ramen Restaurant,Japanese Restaurant,Steakhouse,Café,Seafood Restaurant,Deli / Bodega,Wine Bar


In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Coffee Shop,Café,Wine Shop,Yoga Studio,Gym / Fitness Center,Pizza Place,Bookstore,French Restaurant,Cosmetics Shop,Bar


In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Sandwich Place,Gym,Yoga Studio,Bank,Department Store,Diner,Discount Store,Pharmacy,Donut Shop
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Sandwich Place,Latin American Restaurant,Deli / Bodega,Cocktail Bar,Grocery Store,Gym,Seafood Restaurant
13,Lincoln Square,Plaza,Performing Arts Venue,Café,Concert Hall,Theater,Gym / Fitness Center,Park,Wine Shop,Coffee Shop,Indie Movie Theater
15,Midtown,Hotel,Clothing Store,Coffee Shop,Sporting Goods Shop,Bookstore,Theater,Steakhouse,Cuban Restaurant,Cosmetics Shop,Burger Joint
17,Chelsea,Coffee Shop,Bakery,Art Gallery,Wine Shop,Hotel,American Restaurant,Ice Cream Shop,French Restaurant,Seafood Restaurant,Market
25,Manhattan Valley,Mexican Restaurant,Bar,Coffee Shop,Playground,Thai Restaurant,Indian Restaurant,Vietnamese Restaurant,Pizza Place,Health Food Store,Hawaiian Restaurant
28,Battery Park City,Park,Coffee Shop,Clothing Store,Hotel,Women's Store,Gym,Boat or Ferry,Memorial Site,Playground,Plaza
33,Midtown South,Korean Restaurant,Hotel,Dessert Shop,Hotel Bar,Japanese Restaurant,American Restaurant,Gym / Fitness Center,Coffee Shop,Cosmetics Shop,Burger Joint
37,Stuyvesant Town,Bar,Park,Coffee Shop,Boat or Ferry,Heliport,Farmers Market,Baseball Field,Harbor / Marina,Bistro,Gym / Fitness Center


In [41]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Little Italy,Bakery,Chinese Restaurant,Café,Italian Restaurant,Salon / Barbershop,Sandwich Place,Mediterranean Restaurant,Hotel,Bubble Tea Shop,Ice Cream Shop


### Analysis of Brooklyn neighborhoods, for reference only.

In [42]:
brooklyn_restaurants.sort_values(by='Restaurant count',ascending=False)

,Neighborhood,Restaurant count,Asian Restaurant count
14,Clinton Hill,36,0
56,South Side,29,0
48,Prospect Heights,29,0
15,Cobble Hill,29,0
1,Bay Ridge,26,0
27,Fort Greene,26,0
19,Downtown,25,1
12,Carroll Gardens,23,0
8,Brooklyn Heights,20,2
43,North Side,19,0


In [43]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
print(brooklyn_onehot.shape)
# add neighborhood column back to dataframe
brooklyn_onehot['Neighbourhood'] = brooklyn_venues['Neighborhood'] 
print(brooklyn_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]
#brooklyn_onehot.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
brooklyn_onehot.head()

(2732, 294)
(2732, 295)


,Neighbourhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Stor

In [44]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighbourhood').mean().reset_index()
brooklyn_grouped

,Neighbourhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Stor

In [45]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.08
1       Bubble Tea Shop  0.06
2              Pharmacy  0.06
3  Fast Food Restaurant  0.04
4           Gas Station  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.06
1                  Spa  0.06
2          Pizza Place  0.05
3  American Restaurant  0.04
4                  Bar  0.04


----Bedford Stuyvesant----
           venue  freq
0            Bar  0.09
1    Coffee Shop  0.09
2           Café  0.06
3  Deli / Bodega  0.06
4    Pizza Place  0.06


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.09
1          Donut Shop  0.06
2            Pharmacy  0.06
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.29
1          Playground  0.14
2  Athletics & Sports  0.14
3      Baseball Field  0.14
4                Park  0.14


----Boerum Hill----
                 venue  freq
0       

----Marine Park----
                venue  freq
0    Basketball Court  0.14
1                 Gym  0.14
2  Chinese Restaurant  0.14
3      Baseball Field  0.14
4         Pizza Place  0.14


----Midwood----
           venue  freq
0    Pizza Place  0.31
1       Pharmacy  0.08
2           Café  0.08
3  Grocery Store  0.08
4         Bakery  0.08


----Mill Basin----
                 venue  freq
0          Pizza Place  0.08
1   Chinese Restaurant  0.08
2                 Bank  0.06
3  Japanese Restaurant  0.06
4           Bagel Shop  0.06


----Mill Island----
                           venue  freq
0                           Pool   1.0
1              Accessories Store   0.0
2                Other Nightlife   0.0
3  Paper / Office Supplies Store   0.0
4           Pakistani Restaurant   0.0


----New Lots----
                 venue  freq
0  Fried Chicken Joint  0.10
1        Grocery Store  0.10
2          Pizza Place  0.10
3         Intersection  0.05
4                Plaza  0.05


----North 

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brooklyn_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighbourhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Bubble Tea Shop,Italian Restaurant,Fast Food Restaurant,Donut Shop,Pizza Place,Gas Station,Mobile Phone Shop,Construction & Landscaping
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bar,Bagel Shop,Chinese Restaurant,American Restaurant,Grocery Store,Sushi Restaurant
2,Bedford Stuyvesant,Bar,Coffee Shop,Pizza Place,Deli / Bodega,Café,Fruit & Vegetable Store,Fried Chicken Joint,Gourmet Shop,Gift Shop,Boutique
3,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Pharmacy,Italian Restaurant,Bank,Liquor Store,Coffee Shop,Noodle House
4,Bergen Beach,Harbor / Marina,Park,Athletics & Sports,Baseball Field,Trail,Playground,Farm,Farmers Market,Fast Food Restaurant,Field
5,Boerum Hill,Coffee Shop,Bar,Dance Studio,French Restaurant,Sandwich Place,Furniture / Home Store,Arts & Crafts Store,Bakery,Spa,Cocktail Bar
6,Borough Park,Bank,Pizza Place,Grocery Store,Deli / Bodega,Pharmacy,Café,Chinese Restaurant,Fast Food Restaurant,Bistro,Coffee Shop
7,Brighton Beach,Russian Restaurant,Sushi Restaurant,Pharmacy,Donut Shop,Beach,Eastern European Restaurant,Restaurant,Gourmet Shop,Bank,Mobile Phone Shop
8,Broadway Junction,Gas Station,Donut Shop,Grocery Store,Breakfast Spot,Sandwich Place,Caribbean Restaurant,Nightclub,Lounge,Liquor Store,Coffee Shop
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Park,Diner,Gym / Fitness Center,Cosmetics Shop,Italian Restaurant,Pizza Place,Plaza,Coffee Shop


In [47]:
# set number of clusters
brooklyn_kclusters =8

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighbourhood', 1)

# run k-means clustering
brooklyn_kmeans = KMeans(n_clusters=brooklyn_kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
brooklyn_kmeans.labels_[0:10]

array([6, 7, 7, 6, 0, 7, 6, 6, 6, 7], dtype=int32)

In [48]:
brooklyn_merged = brooklyn_data

# add clustering labels
brooklyn_merged['Cluster Labels'] = brooklyn_kmeans.labels_

brooklyn_merged = brooklyn_merged.join(brooklyn_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,6,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bar,Bagel Shop,Chinese Restaurant,American Restaurant,Grocery Store,Sushi Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,7,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Pharmacy,Italian Restaurant,Bank,Liquor Store,Coffee Shop,Noodle House
2,Brooklyn,Sunset Park,40.645103,-74.010316,7,Mexican Restaurant,Pizza Place,Latin American Restaurant,Bank,Bakery,Pharmacy,Gym,Fried Chicken Joint,Mobile Phone Shop,Grocery Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,6,Coffee Shop,Bar,Pizza Place,Cocktail Bar,Yoga Studio,Furniture / Home Store,Deli / Bodega,French Restaurant,Nail Salon,Garden Center
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Lounge,Bakery,Chinese Restaurant,Bar,Sporting Goods Shop,Electronics Store,Spa,Bus Station,Diner
